# 🏛️ Data Modeling in Data Warehousing

---

## 📦 What is a Data Warehouse?

A **Data Warehouse (DWH)** is a centralized repository designed for reporting, analytics, and business intelligence. It integrates data from various sources and stores it in a **structured**, **historical**, and **query-optimized** format.

---

## 🧠 Key Characteristics of a DWH:
- 💾 **Subject-oriented** (e.g., sales, finance, HR)
- 🕓 **Time-variant** (tracks changes over time)
- 📚 **Non-volatile** (data is read-only once loaded)
- 🔀 **Integrated** (combines data from many sources)

---

## 🧱 Data Modeling Layers

To design a Data Warehouse effectively, we use **three modeling layers**:

| Layer        | Purpose                                                  | Audience                 |
|--------------|-----------------------------------------------------------|--------------------------|
| 🧠 **Conceptual** | High-level business view of entities and relationships   | Business stakeholders     |
| 🧮 **Logical**    | Detailed structure: attributes, keys, relationships       | Data architects, analysts |
| 🛠️ **Physical**   | Actual implementation: data types, indexes, partitions   | Engineers, DBAs           |

---

## 🔍 1️⃣ Conceptual Model

- Abstract view of the **business domain**
- Identifies **main entities** and **high-level relationships**
- No technical details (like data types or constraints)

> 📌 *Example:*
> - Entities: Customer, Product, Order
> - Relationship: Customers place Orders

---

## 🔧 2️⃣ Logical Model

- Refines the conceptual model into detailed **tables and columns**
- Specifies:
  - Primary keys
  - Foreign keys
  - Attribute names
  - Normalization logic
- Still **platform-independent**

> 📌 *Example:*
> - Table: Orders (OrderID, CustomerID, ProductID, Quantity, Date)
> - Foreign key: Orders.CustomerID → Customers.CustomerID

---

## 🛠️ 3️⃣ Physical Model

- Translates logical model into a specific **storage implementation**
- Defines:
  - Data types (INT, STRING, DATE)
  - Partitioning strategy
  - Indexing
  - File format (Parquet, Delta, etc.)
  - Performance optimization (Z-Ordering, bucketing)

> 📌 *Example (Delta Table)*:
> - `CREATE TABLE orders (...) USING DELTA PARTITIONED BY (year,month)`

---

## 🧰 Where Do Tools Like Erwin Fit?

🔧 **Modeling tools** help maintain consistency, automate documentation, and enable collaborative design.  
They are mostly used in the **Conceptual** and **Logical** stages, but many support Physical modeling too.

| Tool        | Purpose & Fit |
|-------------|----------------|
| **Erwin Data Modeler** | Enterprise-grade modeling and governance across conceptual, logical, and physical layers |
| **Lucidchart / dbdiagram.io** | Great for conceptual diagrams or quick visual relationships |
| **SQL DBM** | Cloud-based modeling with native SQL generation |
| **dbt (Data Build Tool)** | Logical/physical model driven by code for analytics engineering, combined with transformation logic |
| **Databricks Unity Catalog** | Tags, schemas, and lineage tracking across physical data models and assets |

**Where to get Ervin?** https://onetakeda.atlassian.net/wiki/spaces/GMSGQDIME/pages/6200001174/Erwin+Data+Modeler+license

> 💡 *Best practice:* Use modeling tools to track lineage, enforce standards, and automate documentation of your warehouse.

---

## 🧩 Why This Matters

| Layer | Purpose |
|-------|---------|
| Conceptual | Align technical and business teams around key concepts |
| Logical | Blueprint for building relationships and enforcing data integrity |
| Physical | Ensures optimal performance and scalability for big data workloads |


# 🌟 Star Schema vs ❄️ Snowflake Schema

---

## 🧠 Why Does Schema Design Matter in Analytics?

In Data Warehousing and Analytics, **schema design** affects:

- 🔍 Query performance  
- 🧩 Data consistency  
- 📊 Report structure  
- 🛠️ Maintainability  

Two common schema models:
- **Star Schema**
- **Snowflake Schema**

---

## 🌟 Star Schema (Denormalized)

### 📌 Characteristics:
- Central **Fact Table** (e.g., sales)  
- Linked directly to **Dimension Tables**  
- Dimension tables are **flattened** (denormalized)
- Simpler and faster for queries

### 🎨 Visual:

---
<pre>
┌──────────────┐       ┌───────────┐       ┌───────┐
│  Customers   │       │  Products │       │ Time  │
└──────────────┘       └───────────┘       └───────┘
         \                 │                 /
          \                │                /
           ▼               ▼               ▼
       ┌──────────────────────────────────┐
       │          📦 Sales (Fact)        │
       └──────────────────────────────────┘
</pre>
---
### 🧾 Example:

**Fact Table: `sales_fact`**

| sale_id | customer_id | product_id | date_id | amount |
|---------|-------------|------------|---------|--------|
| 1       | 101         | 2001       | 20230701| 300.00 |

**Dimension Table: `dim_customer`**

| customer_id | name   | city     | country   |
|-------------|--------|----------|-----------|
| 101         | Alice  | Vienna   | Austria   |


## ❄️ Snowflake Schema (Normalized)

### 📌 Characteristics:
- Fact table still in center  
- Dimensions are **normalized** into sub-tables  
- More joins needed → queries may be slower  
- Good for **storage efficiency**, **data quality**, and **modularity**

### 🎨 Visual:
---
<pre>
             ┌──────────────┐
             │   Region     │
             └─────▲────────┘
                   │
             ┌─────┴──────┐
             │  Country   │
             └─────▲──────┘
                   │
             ┌─────┴────────────┐
             │   Customers      │
             └────────┬─────────┘
                      │
     ┌────────────┐   │   ┌─────────────┐
     │  Category  │   │   │    Time     │
     └─────▲──────┘   │   └──────▲──────┘
           │          │          │
     ┌─────┴──────┐   │   ┌──────┴────────────┐
     │  Products  │───┼──▶│ 📦 Sales (Fact)   │
     └────────────┘       └───────────────────┘  
</pre>
---

### 🧾 Example:

**Dimension Breakdown:**

- `dim_customer` → has `country_id`
- `dim_country` → has `region_id`

**dim_customer**

| customer_id | name   | country_id |
|-------------|--------|------------|
| 101         | Alice  | 10         |

**dim_country**

| country_id | country | region_id |
|------------|---------|-----------|
| 10         | Austria | 1         |

**dim_region**

| region_id | region      |
|-----------|-------------|
| 1         | Europe      |

---

## 🔁 Normalization Theory Recap

| Normal Form | Goal                       | Example Fix                        |
|-------------|----------------------------|-------------------------------------|
| 1NF         | Atomic values only         | Split comma-separated fields       |
| 2NF         | Remove partial dependencies| Separate repeated group values     |
| 3NF         | Remove transitive dep.     | Separate city from customer table  |

> 🧠 *Snowflake schema uses normalization up to 3NF to reduce redundancy and enforce integrity.*

---

## ⚖️ Comparison Table

| Feature              | ⭐ Star Schema     | ❄️ Snowflake Schema |
|----------------------|-------------------|---------------------|
| Joins Required       | Fewer             | More                |
| Performance (Queries)| Faster            | Slower (more joins) |
| Storage Usage        | Higher            | Lower               |
| Maintenance          | Easier            | More complex        |
| Normalization        | Denormalized      | Normalized (3NF)    |
| Use Case             | Reporting, BI     | Complex ETL, compliance |

---

## 🏁 Summary

- **Use Star Schema** when performance is key and simplicity helps reporting teams  
- **Use Snowflake Schema** when storage optimization and relational consistency matter  

In practice, many modern data platforms (including Databricks) support **hybrid approaches**, blending star-style dimensions with normalized lookups.





# 🎯 Granularity in Data Modeling

---

### 🤔 What is Granularity?

Granularity defines the **level of detail** or **depth of data** stored in a fact table within a data warehouse.

- It answers the question:  
  *"What is the most atomic unit of data captured?"*

- The **granularity** determines how **detailed** or **aggregated** your data is.

---

### ⚡ Why is Granularity Important?

- **Too fine granularity**:  
  - ❗ Can cause **large storage** needs and **slow queries**.  
  - Example: Storing every individual click on a website.

- **Too coarse granularity**:  
  - ⚠️ May **lose important details** needed for analysis.  
  - Example: Aggregating daily sales only at the country level, missing store-level insights.

---

### 📊 Examples of Granularity Levels in Sales Data

| Granularity Level                 | Description                                   | Example Fact Record                          |
|---------------------------------|-----------------------------------------------|---------------------------------------------|
| **Transaction-level**            | Each individual sale or event                  | 🛒 Single product sold to a customer at a time |
| **Daily store-level**            | Aggregated sales per store per day             | 🏬 Total daily sales at Store A on July 1       |
| **Monthly region-level**         | Aggregated monthly sales per region             | 🌍 Total sales in the North region in July      |

---

### 📝 Key Considerations

- Granularity **should be consistent** across the fact table.  
- A **well-defined granularity** enables better joins with dimension tables and easier maintenance.  
- The grain must be clear to **avoid mixing data levels** which causes inaccurate analysis.

---

### 🔗 How Granularity Links to Dimensions

The grain of a fact table usually relates to the combination of **foreign keys** linking to dimension tables:

- For example, if granularity is daily sales per store per product, then dimension keys could be:  
  `date_key`, `store_key`, `product_key`

---

### 💡 Summary

> **Defining the correct granularity is foundational for a performant, flexible, and useful data warehouse.**  
> It directly impacts storage size, query performance, and the quality of business insights.

---


# 🔗 Relationships in Data Modeling

---

### 1️⃣ One-to-One (1:1) Relationship

- Each record in **Table A** corresponds to **exactly one** record in **Table B**, and vice versa.
- Rare in data warehouses but useful for splitting tables for security or performance reasons.

**Example:**  
- Employee ↔ Employee Details (e.g., sensitive info stored separately)

| Employee_ID | Name    |  
|-------------|---------|  
| 1           | Alice   |  

| Employee_ID | Social Security Number |  
|-------------|-----------------------|  
| 1           | 123-45-6789           |  

---

### 2️⃣ One-to-Many (1:N) Relationship

- One record in **Table A** can be linked to **many records** in **Table B**.
- This is the most common relationship in data warehouses.

**Example:**  
- Customer(dim) → Orders(fact)  
- A single customer can have multiple orders.

| Customer_ID | Name      |  
|-------------|-----------|  
| 1001        | John Doe  |  

| Order_ID | Customer_ID | Order_Date |  
|----------|-------------|------------|  
| 5001     | 1001        | 2025-07-01 |  
| 5002     | 1001        | 2025-07-03 |  

---

### 3️⃣ Many-to-Many (M:N) Relationship

- Many records in **Table A** can relate to many records in **Table B**.
- Usually modeled by introducing a **junction (bridge) table**.

**Example:**  
- Students ↔ Courses  
- A student can enroll in many courses, and a course can have many students.

| Student_ID | Name      |  
|------------|-----------|  
| 1          | Alice     |  

| Course_ID | Course_Name |  
|-----------|-------------|  
| 101       | Math        |  

| Enrollment (Junction Table) |  
|-----------------------------|  
| Student_ID | Course_ID      |  
| 1          | 101            |  

---

### 🔄 Summary of Relationships

| Relationship   | Description                     | Data Modeling Example          |
|----------------|--------------------------------|-------------------------------|
| One-to-One     | 1 record ↔ 1 record            | Employee ↔ Employee Details    |
| One-to-Many    | 1 record ↔ Many records        | Customer → Orders              |
| Many-to-Many   | Many records ↔ Many records    | Students ↔ Courses (via Enrollment) |

---

### 💡 Pro Tip

- **Foreign keys** implement these relationships physically.
- Properly defining relationships ensures **data integrity** and **efficient joins**.

---



# ⚙️ Data Load Strategies in Data Warehousing

---

## 1️⃣ Append-Only Load

- New data rows are **only appended** to the target table.
- No rows are updated or deleted.
- Often combined with a **marker column** to indicate the type of operation:
  - `'I'` = Insert
  - `'U'` = Update
  - `'D'` = Delete (soft delete marker)

**Use case:**  
Audit logs, event streams where history is preserved.

---

## 2️⃣ Truncate / Full Load

- Target table is **completely emptied (truncated)** before loading fresh data.
- Simple but expensive and usually only suitable for small datasets or staging layers.

**Use case:**  
Initial data load or when source data refreshes completely.

---

## 3️⃣ Slowly Changing Dimension Type 1 (SCD1)

- Overwrites existing records with new data.
- No history is kept.
- Simple and efficient for attributes where history is not important.

**Example:**  
Updating a customer’s address to the latest one.

---

## 4️⃣ Custom SCD1,5 with Active Flag (Custom Approach)

- Similar to SCD1 but **no deletes**.
- Instead of deleting, update an **active flag** from `'Y'` → `'N'`.
- Keeps data "soft deleted" for audit and historical reference.
- Only one record is marked active (`'Y'`) per business key at a time.

**Benefits:**  
- Easy to implement and query.
- Supports some history without complex SCD2 mechanics.
- Avoids physical deletes improving data safety.

**Example schema snippet:**

| customer_id | name     | address      | active_flag | last_updated          |
|-------------|----------|--------------|-------------|-----------------------|
| 1001        | John Doe | 123 Main St  | Y           | 2025-07-22 12:00:00   |
| 1001        | John Doe | 456 Oak Ave  | N           | 2025-01-15 08:30:00   |
---

## 5️⃣ Slowly Changing Dimension Type 2 (SCD2)

- Keeps full history of changes by creating **new records** for changes.
- Adds columns like `effective_date`, `end_date`, and `current_flag` to track active record.
- Supports historical analysis.

**Example:**  
Tracking changes in customer’s health plan status over time.


---

### 🔄 Summary Table of Load Strategies

| Strategy                 | History Preserved? | Typical Use Case                 | Notes                              |
|--------------------------|--------------------|--------------------------------|----------------------------------|
| Append-Only              | No                 | Audit logs, event streams       | Marker column tracks operation   |
| Truncate / Full Load     | No                 | Initial loads, small datasets   | Expensive for big tables         |
| SCD Type 1               | No                 | Overwrite attributes            | Simple, no history               |
| SCD Type 1,5  | Partial          | Soft deletes, light history     | No physical deletes, simple logic|
| SCD Type 2               | Yes                | Full history tracking           | Complex, multiple records per key|


---

### 💡 Important in our pipelines

- We typically use **Custom SCD1 with Active Flag** for easy auditability and soft deletes.
- Delta Live Tables (DLT) and Spark SQL `MERGE` statements make implementing these patterns efficient and clear.
- Adding **row keys, audit columns, and standardization** functions ensures clean, consistent data loads.

---


# 🔁 Delta Live Tables (DLT) & Change Data Capture (CDC)

Delta Live Tables offers a powerful and simplified way to process **Change Data Capture (CDC)** using **declarative syntax**, removing the complexity of manual `MERGE` statements.

---

## 💡 Why This Matters

Traditional CDC implementations often rely on complex `MERGE` statements that:
- Fail with out-of-order data
- Struggle with deletes/truncates
- Are hard to scale or maintain

With **DLT’s built-in CDC capabilities**, you can now declaratively apply changes, deduplicate, sequence, and even handle slowly changing dimensions (SCDs).

### ✅ `AUTO CDC INTO` (SQL)

```sql
CREATE FLOW cdc_pipeline AS AUTO CDC INTO target_table
FROM source_table
KEYS (business_key)
APPLY AS DELETE WHEN operation = 'DELETE'
SEQUENCE BY event_ts
STORED AS SCD TYPE 2
TRACK HISTORY ON *;


```
### ✅ `AUTO CDC INTO` (PYTHON)
```import dlt

dlt.create_auto_cdc_flow(
    target="prod.sales_orders",
    source="raw.cdc_orders",
    keys=["order_id"],
    sequence_by="event_ts",
    stored_as_scd_type="SCD2",
    apply_as_deletes="operation = 'DELETE'"
)
```


---
**For more detailed information pelase check documentation:** https://docs.databricks.com/aws/en/dlt/cdc
